<a href="https://colab.research.google.com/github/pahwa1902/LearMachineLearning/blob/master/DisasterTweet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import re
import tensorflow as tf
import tensorflow_hub as hub
from keras.layers import LSTM
from tensorflow.keras import layers
tf.random.set_random_seed(1)
import tensorflow_datasets as tfds
from gensim.models import KeyedVectors
import numpy as np
tf.random.set_random_seed(1)
import nltk
import string

In [0]:
! ls '/content/drive/My Drive/nlp-getting-started'

In [0]:
data_train = pd.read_csv('/content/drive/My Drive/nlp-getting-started/train.csv')

In [0]:
training_data = pd.DataFrame(data_train)

In [0]:
training_data.tail()

In [0]:
training_data['text'] = training_data['text'].apply(lambda x : re.sub(u'(\s)@\w+', u'', x))
training_data['text'] = training_data['text'].apply(lambda x : re.sub(u'#', u'', x))
training_data['text'] = training_data['text'].apply(lambda x : re.sub(u'http\S+', u'', x))
training_data['text'] = training_data['text'].apply(lambda x : re.sub(u'[?|$|.|!|=>|&|;|:)|+|@|*|-]',u'',x))
training_data['text'] = training_data['text'].apply(lambda x : re.sub(u'[0-9]',u'',x))
training_data['text'] = training_data['text'].apply(lambda x : re.sub(u'(\w+):',u'',x))
training_data['text'] = training_data['text'].apply(lambda x : re.sub(u'(end\>)',u'',x))

training_data['text'] = training_data['text'].apply(lambda x : x.lower())
#training_data['text'] = training_data['text'].apply(lambda x : re.sub(u'(\'|\,|\(|\))',u'',x))

#training_data['text'] = training_data['text'].apply(lambda x : re.sub(u'(\/)',u' ',x))
#training_data['text'] = training_data['text'].apply(lambda x : re.sub(u'()',u' ',x))

In [114]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
training_data['text'] = training_data['text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

In [0]:
training_data['text']

In [0]:
table = str.maketrans('', '', string.punctuation)

In [0]:
training_data['text'] = [training_data['text'][row].translate(table) for row in range(len(training_data['text']))]

In [0]:
embedding = "https://tfhub.dev/google/nnlm-en-dim128/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)

In [0]:
summ=[]
for i in range(len(training_data)):
  a = training_data['text'].iloc[i]
  summ+=a.split(' ')

In [0]:
print(set(summ))

In [0]:
training_data = training_data.reindex(columns=training_data.columns.tolist()+['frequency'])

In [0]:
def count(a,b):
  count=0
  for i in b.split(' '):
    if (i==a):
      count+=1
  return count

In [0]:
frequency={}
for i in set(summ):
  j=0
  training_data['frequency']=training_data['text'].apply(lambda x:count(i,x))
  frequency[i]=sum(training_data['frequency'])


In [0]:
frequency

In [0]:
def delete_freq(a):
  return [i for i,u in frequency.items() if u==a]

In [0]:
def delete(a):
  for i in a:
    del frequency[i]
  return frequency

In [0]:
frequency = delete(delete_freq(1))
frequency = delete(delete_freq(2))

In [0]:
sorted_frequency = {k: v for k, v in sorted(frequency.items(),reverse=True, key=lambda item: item[1])}

In [0]:
del sorted_frequency['rt']
del sorted_frequency['']

In [0]:
del sorted_frequency['\x89û']

In [0]:
training_data['text'] = training_data['text'].apply(lambda x : " ".join(x for x in x.split() if x in sorted_frequency))

In [0]:
def char2int(b):
  vocab = len(set(b))
  dict_vocab = {}
  a = list(set(b))
  for i in range(vocab):
    dict_vocab[a[i]] = i
  return dict_vocab,vocab

In [0]:
#char2id = {u:i for i,u in char2int[0].items()}
training_data = training_data.reindex(columns=training_data.columns.tolist()+['text2int'])


In [0]:
encoder = char2int(summ)[0]
vocab_size = char2int(summ)[1]

In [0]:
training_data['text2int'] = training_data['text'].apply(lambda x : np.array([encoder[c] for c in x.split(' ')]))

In [0]:
training_data.head()

In [0]:
training_data = training_data.reindex(columns=training_data.columns.tolist()+['length'])
training_data['length']=training_data['text2int'].apply(lambda x : len(x))
pad_size = max(training_data['length'])
training_data['text2int'] = training_data['text2int'].apply(lambda x : np.array(x).tolist() + np.zeros(pad_size-len(x)).tolist())

In [0]:

train_data = training_data.sample(frac=0.8)
training_data_sample = train_data['text2int']
train_label = train_data['target']
validate_data = training_data.drop(train_data.index,axis=0)
validate_data_sample = validate_data['text2int']
validate_label = validate_data['target']

In [0]:
train_data_mod = training_data.iloc[:5700]['text2int']
train_label_mod = label[:5700]
validate_data_mod = training_data.iloc[5700:]['text2int']
validate_label_mod = label[5700:]

In [0]:
train_data_mod=training_data_sample.tolist()
validate_data_mod = validate_data_sample.tolist()

In [0]:
train_data_mod

In [0]:
columns=[]
for i in range(54):
  columns.append('v'+str(i))

In [0]:
train_dataset_mod = tf.data.Dataset.from_tensor_slices(train_data_mod)

In [0]:
iterator = train_dataset_mod.make_one_shot_iterator()
next_element = iterator.get_next()

In [0]:
with tf.Session() as sess:
    for i in range(1):
        val = sess.run(next_element)
        print(val)

In [0]:
label_dataset_mod = tf.data.Dataset.from_tensor_slices((train_label))

In [0]:
train_label_dataset = tf.data.Dataset.zip((train_dataset_mod,label_dataset_mod)).shuffle(1000).repeat(5).batch(64)

In [0]:
validate_dataset_mod = tf.data.Dataset.from_tensor_slices(validate_data_mod)

In [0]:
label_validate_mod = tf.data.Dataset.from_tensor_slices((validate_label))

In [0]:
validate_label_dataset = tf.data.Dataset.zip((validate_dataset_mod,validate_dataset_mod)).batch(64)

In [0]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_data,train_label))
#train_label = tf.data.Dataset.from_tensor_slices((label))
train_dataset = train_dataset.shuffle(len(train_data))
#train_label = train_label.shuffle(len(train_data)).batch(64)

validate_dataset = tf.data.Dataset.from_tensor_slices((validate_data,validate_label))


In [0]:
def encode(data,label):
  encoded_data = np.array([encoder[c] for c in data.split(' ')])
  return encoded_data,label

In [0]:
def encode_fn(text,label):
  return tf.py_function(encode,inp=[text,label],Tout=(tf.int32,tf.int32))

In [0]:
train_dataset = train_dataset.map(encode_fn)
validate_dataset = validate_dataset.map(encode_fn)

In [0]:
train_data_new = train_dataset.shuffle(1000).padded_batch(64,padded_shapes=([-1],[]))

In [0]:
validate_data_new = validate_dataset.padded_batch(64,padded_shapes=([-1],[]))

In [0]:
model = tf.keras.Sequential();
model.add(tf.keras.layers.Embedding(vocab_size,64));
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(1)));
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

In [0]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [0]:
history = model.fit(train_label_dataset,epochs=10,verbose=1,validation_data=validate_label_dataset)